# Start code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [4]:

train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Create model

In [5]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\nilsb\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nilsb\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

# Train model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
# just to double check cuda is working
print(device.type)

epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

cuda
Epoch 0, iter 0, loss: 14992.4453125
Epoch 0, iter 10, loss: 14904.0361328125
Epoch 0, iter 20, loss: 13910.798828125
Epoch 0, iter 30, loss: 13742.23828125
Epoch 0, iter 40, loss: 13050.3916015625
Epoch 0, iter 50, loss: 12959.2099609375
Epoch 0, iter 60, loss: 13736.212890625
Epoch 0, iter 70, loss: 10991.537109375
Epoch 0, iter 80, loss: 12349.6279296875
Epoch 0, iter 90, loss: 11734.8388671875
Epoch 0, iter 100, loss: 11828.654296875
Epoch 0, iter 110, loss: 10446.9560546875
Epoch 0, iter 120, loss: 10558.603515625
Epoch 0, iter 130, loss: 10208.35546875
Epoch 0, iter 140, loss: 9215.1796875
Epoch 0, iter 150, loss: 9455.9482421875
Epoch 0, iter 160, loss: 8967.2255859375
Epoch 0, iter 170, loss: 8589.30078125
Epoch 0, iter 180, loss: 8498.0732421875
Epoch 0, iter 190, loss: 8463.216796875
Epoch 0, iter 200, loss: 7509.89306640625
Epoch 0, iter 210, loss: 7332.21875
Epoch 0, iter 220, loss: 7356.29833984375
Epoch 0, iter 230, loss: 7371.41650390625
Epoch 0, iter 240, loss: 634

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")